In [1]:
import pandas as pd
import numpy as np
from dash import Dash, dcc, html, Input, Output
import plotly.express as px
import flask
from fontTools.misc.cython import returns

/Users/joshua/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
# max_dt = pd.to_datetime("1900-01-01 12:00:00")
# min_dt = pd.to_datetime("2099-01-01 12:00:00")
# csv_dict = {
#     "basal": pd.read_csv("IT 30 ENG/Insulin data/basal_data_1.csv", skiprows=1)
#     , "bolus": pd.read_csv("IT 30 ENG/Insulin data/bolus_data_1.csv", skiprows=1)
#     , "insulin": pd.read_csv("IT 30 ENG/Insulin data/insulin_data_1.csv", skiprows=1)
#     , "alarms": pd.read_csv("IT 30 ENG/alarms_data_1.csv", skiprows=1)
#     , "bg": pd.read_csv("IT 30 ENG/bg_data_1.csv", skiprows=1)
#     , "cgm": pd.read_csv("IT 30 ENG/cgm_data_1.csv", skiprows=1)
# }


{
    "basal": ["basal_data_1.csv"],
    "bolus": ["bolus_data_1.csv"],
    "insulin": ["insulin_data_1.csv"],
    "alarms": [],
    "bg": [],
    "cgm": ["cgm_data_1.csv", "cgm_data_3.csv", "cgm_data_2.csv"]
}

# for key, value in csv_dict.items():
#     value["Timestamp"] = pd.to_datetime(value["Timestamp"])
#     value = value.set_index("Timestamp").drop("Serial Number", axis=1)
#     value = value.add_prefix(f"{key}_")
#     if max_dt < value.index.max():
#         max_dt = value.index.max()
#     if min_dt > value.index.min():
#         min_dt = value.index.min()
#     csv_dict[key] = value

In [3]:
# df = pd.DataFrame({'DateTime': pd.date_range(start=min_dt, end=max_dt, freq='1min')})
# df = df.set_index('DateTime')
# 
# for key, value in csv_dict.items():
#     df = df.join(value)
# display(df)

#df.to_csv("PumpPerfect_compiled.csv")

# df[~df['this_column'].isna()]
# df.query('A > B')

In [ ]:
@app.route('/upload', methods=['POST'])
def upload_file():
    # Define the expected keys for the uploaded files
    expected_keys = ['basal', 'bolus', 'insulin', 'alarms', 'bg', 'cgm']
    csv_dict = {}

    # Check for uploaded files
    for key in expected_keys:
        if key in request.files:
            file = request.files[key]

            # Check if the file is not empty
            if file.filename == '':
                return jsonify({'error': f'No file selected for {key}'}), 400

            # Read the file content into a DataFrame
            try:
                file_content = file.stream.read().decode("utf-8")
                df = pd.read_csv(StringIO(file_content), skiprows=1)
                csv_dict[key] = df  # Store the DataFrame in the dictionary

            except Exception as e:
                return jsonify({'error': f'Error processing file {key}: {str(e)}'}), 500

    # Return the dictionary with DataFrames converted to JSON
    process_csv(jsonify({key: df.to_dict(orient='records') for key, df in csv_dict.items()}))
    
    # return jsonify({key: df.to_dict(orient='records') for key, df in csv_dict.items()})


In [4]:
def process_csv(csv_dict):
    
    max_dt = pd.to_datetime("1900-01-01 12:00:00")
    min_dt = pd.to_datetime("2099-01-01 12:00:00")

    for key, value in csv_dict.items():
        value["Timestamp"] = pd.to_datetime(value["Timestamp"])
        value = value.set_index("Timestamp").drop("Serial Number", axis=1)
        value = value.add_prefix(f"{key}_")
        if max_dt < value.index.max():
            max_dt = value.index.max()
        if min_dt > value.index.min():
            min_dt = value.index.min()
        csv_dict[key] = value

    df = pd.DataFrame({'DateTime': pd.date_range(start=min_dt, end=max_dt, freq='1min')})
    df = df.set_index('DateTime')

    for key, value in csv_dict.items():
        df = df.join(value)

    df["basal_Insulin Delivered (U)"] = df["basal_Rate"] / 60  # df["basal_Duration (minutes)"] *
    df["basal_Insulin Delivered (U)"] = df["basal_Insulin Delivered (U)"].ffill()
    df['bolus_Trigger'] = np.where(df['bolus_Carbs Input (g)'] == 0, 'Automatic', 'Manual')
    df["bolus_Insulin Delivered (U)"] = df["bolus_Insulin Delivered (U)"].fillna(0)
    df["cgm_CGM Glucose Value (mmol/l)"] = df["cgm_CGM Glucose Value (mmol/l)"].interpolate(method='time')
    df["Time"] = (df.index).time

    return df

In [5]:
# df["basal_Insulin Delivered (U)"] = df["basal_Rate"] / 60  # df["basal_Duration (minutes)"] *
# df["basal_Insulin Delivered (U)"] = df["basal_Insulin Delivered (U)"].ffill()
# df['bolus_Trigger'] = np.where(df['bolus_Carbs Input (g)'] == 0, 'Automatic', 'Manual')
# df["bolus_Insulin Delivered (U)"] = df["bolus_Insulin Delivered (U)"].fillna(0)
# df["cgm_CGM Glucose Value (mmol/l)"] = df["cgm_CGM Glucose Value (mmol/l)"].interpolate(method='time')
# df["Time"] = (df.index).time
# display(df)

In [6]:

# print(datafram.columns)

# # Select the ones you want
# df = df[
#     ['basal_Insulin Delivered (U)', 'bolus_Insulin Delivered (U)', 'bolus_Trigger', 'cgm_CGM Glucose Value (mmol/l)']]

In [7]:
# Initialize the Dash app
app = Dash(__name__)
df = process_csv()
 
# Layout of the Dash app
app.layout = html.Div([
    html.H1("Time Series Line Plot"),

    # Date picker for selecting date range (optional)

    dcc.DatePickerRange(
        id='date-range-picker',
        start_date=df.index.min(),
        end_date=df.index.max()
    ),

    # Line plot
    dcc.Graph(id='line-plot')
])


# Callback to update the line plot based on date range selection
@app.callback(
    Output('line-plot', 'figure'),
    Input('date-range-picker', 'start_date'),
    Input('date-range-picker', 'end_date')
)
def update_line_plot(start_date, end_date):
    # Filter data based on the selected date range
    filtered_df = df.loc[start_date:end_date]

    # Create line plot with Plotly Express
    fig = px.line(
        filtered_df,
        x=filtered_df.index,
        y='basal_Insulin Delivered (U)',
        title="Time Series Line Plot",
        labels={"x": "Date", "Value": "Value"}
    )

    fig.update_layout(
        xaxis_title="Date",
        yaxis_title="Value"
    )
    return fig


if __name__ == '__main__':
    app.run_server(debug=True)

FileNotFoundError: [Errno 2] No such file or directory: 'IT 30 ENG/Insulin data/basal_data_1.csv'

In [8]:
# Specify the series to plot directly in the code
selected_series = ['basal_Insulin Delivered (U)', 'cgm_CGM Glucose Value (mmol/l)']  # Choose specific columns to plot

#'basal_Insulin Delivered (U)','bolus_Insulin Delivered (U)', 'bolus_Trigger', 'cgm_CGM Glucose Value (mmol/l)'

# Initialize the Dash app
app = Dash(__name__)

# Layout of the Dash app
app.layout = html.Div([
    html.H1("Pre-Selected Time Series Line Plot"),

    # Date picker for selecting date range
    dcc.DatePickerRange(
        id='date-range-picker',
        start_date=df.index.min(),
        end_date=df.index.max()
    ),

    # Line plot
    dcc.Graph(id='line-plot')
])


# Callback to update the line plot based on selected date range
@app.callback(
    Output('line-plot', 'figure'),
    Input('date-range-picker', 'start_date'),
    Input('date-range-picker', 'end_date')
)
def update_line_plot(start_date, end_date):
    # Filter data based on selected date range and specific series
    filtered_df = df.loc[start_date:end_date, selected_series]

    # Reshape data for Plotly Express compatibility
    filtered_df_long = filtered_df.reset_index().melt(id_vars='DateTime', var_name='Series', value_name='Value')

    # Create a line plot with Plotly Express
    fig = px.line(
        filtered_df_long,
        x='DateTime',
        y='Value',
        color='Series',
        title="Time Series Line Plot with Pre-Selected Series",
        labels={"index": "Date", "Value": "Value"}
    )

    fig.update_layout(
        xaxis_title="Date",
        yaxis_title="Value"
    )
    return fig


# Run the Dash app
if __name__ == '__main__':
    app.run_server(debug=True)


In [9]:
# Specify the series to plot in each figure
selected_series = {
    "Figure 1": ["basal_Insulin Delivered (U)"],  # First figure with two series
    "Figure 2": ["bolus_Insulin Delivered (U)"],  # Second figure with one series
    "Figure 3": ["cgm_CGM Glucose Value (mmol/l)"]  # Third figure with another combination
}

#'basal_Insulin Delivered (U)','bolus_Insulin Delivered (U)', 'bolus_Trigger', 'cgm_CGM Glucose Value (mmol/l)'

# Initialize the Dash app
app = Dash(__name__)

# Layout of the Dash app
app.layout = html.Div([
    html.H1("Multiple Line Plots with Shared Date Picker"),

    # Date picker for selecting date range
    dcc.DatePickerRange(
        id='date-range-picker',
        start_date=df.index.min(),
        end_date=df.index.max()
    ),

    # Multiple line plots
    html.Div([
        dcc.Graph(id='line-plot-1'),
        dcc.Graph(id='line-plot-2'),
        dcc.Graph(id='line-plot-3')
    ])
])


# Callback to update each line plot based on selected date range
@app.callback(
    [Output('line-plot-1', 'figure'),
     Output('line-plot-2', 'figure'),
     Output('line-plot-3', 'figure')],
    [Input('date-range-picker', 'start_date'),
     Input('date-range-picker', 'end_date')]
)
def update_line_plots(start_date, end_date):
    # Create figures for each plot
    figures = []
    for fig_title, series_list in selected_series.items():
        # Filter data for the specified date range and series list
        filtered_df = df.loc[start_date:end_date, series_list]

        # Reshape data to long format for Plotly Express
        filtered_df_long = filtered_df.reset_index().melt(id_vars='DateTime', var_name='Series', value_name='Value')

        # Create a line plot
        fig = px.line(
            filtered_df_long,
            x='DateTime',
            y='Value',
            color='Series',
            title=fig_title,
            labels={"index": "Date", "Value": "Value"}
        )

        fig.update_layout(
            xaxis_title="Date",
            yaxis_title="Value"
        )

        # Add the figure to the list
        figures.append(fig)

    return figures


# Run the Dash app
if __name__ == '__main__':
    app.run_server(debug=True)


In [ ]:
# Specify the series to plot directly in the code
selected_series = ['basal_Insulin Delivered (U)', 'bolus_Insulin Delivered (U)']  # Choose specific columns to plot

#'basal_Insulin Delivered (U)','bolus_Insulin Delivered (U)', 'bolus_Trigger', 'cgm_CGM Glucose Value (mmol/l)'

# Initialize the Dash app
app = Dash(__name__)

# Layout of the Dash app
app.layout = html.Div([
    html.H1("Pre-Selected Time Series Line Plot"),

    # Date picker for selecting date range
    dcc.DatePickerRange(
        id='date-range-picker',
        start_date=df.index.min(),
        end_date=df.index.max()
    ),

    # Line plot
    dcc.Graph(id='line-plot')
])


# Callback to update the line plot based on selected date range
@app.callback(
    Output('line-plot', 'figure'),
    Input('date-range-picker', 'start_date'),
    Input('date-range-picker', 'end_date')
)
def update_line_plot(start_date, end_date):
    # Filter data based on selected date range and specific series
    filtered_df = df.loc[start_date:end_date, selected_series]

    # Reshape data for Plotly Express compatibility
    filtered_df_long = filtered_df.reset_index().melt(id_vars='DateTime', var_name='Series', value_name='Value')

    filtered_df_long = filtered_df_long.set_index("DateTime").join(df['bolus_Trigger'])

    # Add a color column based on the bolus_Trigger condition
    filtered_df_long['Color'] = np.where(
        (filtered_df_long['Series'] == 'bolus_Insulin Delivered (U)') & (
                filtered_df_long['bolus_Trigger'].values == "Automatic"),
        'red', 'blue'
    )

    filtered_df_long = filtered_df_long.drop("bolus_Trigger", axis=1).reset_index()
    display(filtered_df_long[filtered_df_long["Color"] == 'red'])

    # Create a line plot with Plotly Express
    fig = px.line(
        filtered_df_long,
        x='DateTime',
        y='Value',
        color='Series',
        title="Time Series Line Plot with Pre-Selected Series",
        labels={"index": "Date", "Value": "Value"},
        color_discrete_sequence=filtered_df_long['Color']  # Use the conditional colors
    )

    fig.update_layout(
        xaxis_title="Date",
        yaxis_title="Value"
    )
    return fig


# Run the Dash app
if __name__ == '__main__':
    app.run_server(debug=True)


,DateTime,Series,Value,Color
43936,2024-06-02 12:17:00,bolus_Insulin Delivered (U),0.400,red
44172,2024-06-02 16:13:00,bolus_Insulin Delivered (U),0.547,red
44368,2024-06-02 19:29:00,bolus_Insulin Delivered (U),0.389,red
45344,2024-06-03 11:42:00,bolus_Insulin Delivered (U),0.326,red
45383,2024-06-03 12:21:00,bolus_Insulin Delivered (U),0.410,red
...,...,...,...,...
85737,2024-07-01 11:00:00,bolus_Insulin Delivered (U),0.395,red
85961,2024-07-01 14:44:00,bolus_Insulin Delivered (U),0.382,red
86036,2024-07-01 15:59:00,bolus_Insulin Delivered (U),0.304,red
86301,2024-07-01 20:24:00,bolus_Insulin Delivered (U),0.427,red


,DateTime,Series,Value,Color
43936,2024-06-02 12:17:00,bolus_Insulin Delivered (U),0.400,red
44172,2024-06-02 16:13:00,bolus_Insulin Delivered (U),0.547,red
44368,2024-06-02 19:29:00,bolus_Insulin Delivered (U),0.389,red
45344,2024-06-03 11:42:00,bolus_Insulin Delivered (U),0.326,red
45383,2024-06-03 12:21:00,bolus_Insulin Delivered (U),0.410,red
...,...,...,...,...
85737,2024-07-01 11:00:00,bolus_Insulin Delivered (U),0.395,red
85961,2024-07-01 14:44:00,bolus_Insulin Delivered (U),0.382,red
86036,2024-07-01 15:59:00,bolus_Insulin Delivered (U),0.304,red
86301,2024-07-01 20:24:00,bolus_Insulin Delivered (U),0.427,red


,DateTime,Series,Value,Color
43936,2024-06-02 12:17:00,bolus_Insulin Delivered (U),0.400,red
44172,2024-06-02 16:13:00,bolus_Insulin Delivered (U),0.547,red
44368,2024-06-02 19:29:00,bolus_Insulin Delivered (U),0.389,red
45344,2024-06-03 11:42:00,bolus_Insulin Delivered (U),0.326,red
45383,2024-06-03 12:21:00,bolus_Insulin Delivered (U),0.410,red
...,...,...,...,...
85737,2024-07-01 11:00:00,bolus_Insulin Delivered (U),0.395,red
85961,2024-07-01 14:44:00,bolus_Insulin Delivered (U),0.382,red
86036,2024-07-01 15:59:00,bolus_Insulin Delivered (U),0.304,red
86301,2024-07-01 20:24:00,bolus_Insulin Delivered (U),0.427,red


,DateTime,Series,Value,Color
43936,2024-06-02 12:17:00,bolus_Insulin Delivered (U),0.400,red
44172,2024-06-02 16:13:00,bolus_Insulin Delivered (U),0.547,red
44368,2024-06-02 19:29:00,bolus_Insulin Delivered (U),0.389,red
45344,2024-06-03 11:42:00,bolus_Insulin Delivered (U),0.326,red
45383,2024-06-03 12:21:00,bolus_Insulin Delivered (U),0.410,red
...,...,...,...,...
85737,2024-07-01 11:00:00,bolus_Insulin Delivered (U),0.395,red
85961,2024-07-01 14:44:00,bolus_Insulin Delivered (U),0.382,red
86036,2024-07-01 15:59:00,bolus_Insulin Delivered (U),0.304,red
86301,2024-07-01 20:24:00,bolus_Insulin Delivered (U),0.427,red


,DateTime,Series,Value,Color
43936,2024-06-02 12:17:00,bolus_Insulin Delivered (U),0.400,red
44172,2024-06-02 16:13:00,bolus_Insulin Delivered (U),0.547,red
44368,2024-06-02 19:29:00,bolus_Insulin Delivered (U),0.389,red
45344,2024-06-03 11:42:00,bolus_Insulin Delivered (U),0.326,red
45383,2024-06-03 12:21:00,bolus_Insulin Delivered (U),0.410,red
...,...,...,...,...
85737,2024-07-01 11:00:00,bolus_Insulin Delivered (U),0.395,red
85961,2024-07-01 14:44:00,bolus_Insulin Delivered (U),0.382,red
86036,2024-07-01 15:59:00,bolus_Insulin Delivered (U),0.304,red
86301,2024-07-01 20:24:00,bolus_Insulin Delivered (U),0.427,red


,DateTime,Series,Value,Color
43936,2024-06-02 12:17:00,bolus_Insulin Delivered (U),0.400,red
44172,2024-06-02 16:13:00,bolus_Insulin Delivered (U),0.547,red
44368,2024-06-02 19:29:00,bolus_Insulin Delivered (U),0.389,red
45344,2024-06-03 11:42:00,bolus_Insulin Delivered (U),0.326,red
45383,2024-06-03 12:21:00,bolus_Insulin Delivered (U),0.410,red
...,...,...,...,...
85737,2024-07-01 11:00:00,bolus_Insulin Delivered (U),0.395,red
85961,2024-07-01 14:44:00,bolus_Insulin Delivered (U),0.382,red
86036,2024-07-01 15:59:00,bolus_Insulin Delivered (U),0.304,red
86301,2024-07-01 20:24:00,bolus_Insulin Delivered (U),0.427,red


,DateTime,Series,Value,Color
43936,2024-06-02 12:17:00,bolus_Insulin Delivered (U),0.400,red
44172,2024-06-02 16:13:00,bolus_Insulin Delivered (U),0.547,red
44368,2024-06-02 19:29:00,bolus_Insulin Delivered (U),0.389,red
45344,2024-06-03 11:42:00,bolus_Insulin Delivered (U),0.326,red
45383,2024-06-03 12:21:00,bolus_Insulin Delivered (U),0.410,red
...,...,...,...,...
85737,2024-07-01 11:00:00,bolus_Insulin Delivered (U),0.395,red
85961,2024-07-01 14:44:00,bolus_Insulin Delivered (U),0.382,red
86036,2024-07-01 15:59:00,bolus_Insulin Delivered (U),0.304,red
86301,2024-07-01 20:24:00,bolus_Insulin Delivered (U),0.427,red


In [11]:
def SummarizeMetric(df, col):
    df_summary = df.groupby('Time').agg(
        Median=(col, 'median'),
        Q1=(col, lambda x: x.quantile(0.25)),
        Q3=(col, lambda x: x.quantile(0.75)),
        D1=(col, lambda x: x.quantile(0.1)),
        D9=(col, lambda x: x.quantile(0.9)),
        Min=(col, 'min'),
        Max=(col, 'max'),
    ).reset_index()

    return df_summary

In [12]:
df["Time"] = (df.index).time

In [ ]:
df_glucose = SummarizeMetric(df, 'cgm_CGM Glucose Value (mmol/l)')
display(df_glucose)

,DateTime,Series,Value,Color
43936,2024-06-02 12:17:00,bolus_Insulin Delivered (U),0.400,red
44172,2024-06-02 16:13:00,bolus_Insulin Delivered (U),0.547,red
44368,2024-06-02 19:29:00,bolus_Insulin Delivered (U),0.389,red
45344,2024-06-03 11:42:00,bolus_Insulin Delivered (U),0.326,red
45383,2024-06-03 12:21:00,bolus_Insulin Delivered (U),0.410,red
...,...,...,...,...
85737,2024-07-01 11:00:00,bolus_Insulin Delivered (U),0.395,red
85961,2024-07-01 14:44:00,bolus_Insulin Delivered (U),0.382,red
86036,2024-07-01 15:59:00,bolus_Insulin Delivered (U),0.304,red
86301,2024-07-01 20:24:00,bolus_Insulin Delivered (U),0.427,red


,DateTime,Series,Value,Color
43936,2024-06-02 12:17:00,bolus_Insulin Delivered (U),0.400,red
44172,2024-06-02 16:13:00,bolus_Insulin Delivered (U),0.547,red
44368,2024-06-02 19:29:00,bolus_Insulin Delivered (U),0.389,red
45344,2024-06-03 11:42:00,bolus_Insulin Delivered (U),0.326,red
45383,2024-06-03 12:21:00,bolus_Insulin Delivered (U),0.410,red
...,...,...,...,...
85737,2024-07-01 11:00:00,bolus_Insulin Delivered (U),0.395,red
85961,2024-07-01 14:44:00,bolus_Insulin Delivered (U),0.382,red
86036,2024-07-01 15:59:00,bolus_Insulin Delivered (U),0.304,red
86301,2024-07-01 20:24:00,bolus_Insulin Delivered (U),0.427,red


,DateTime,Series,Value,Color
43936,2024-06-02 12:17:00,bolus_Insulin Delivered (U),0.400,red
44172,2024-06-02 16:13:00,bolus_Insulin Delivered (U),0.547,red
44368,2024-06-02 19:29:00,bolus_Insulin Delivered (U),0.389,red
45344,2024-06-03 11:42:00,bolus_Insulin Delivered (U),0.326,red
45383,2024-06-03 12:21:00,bolus_Insulin Delivered (U),0.410,red
...,...,...,...,...
85737,2024-07-01 11:00:00,bolus_Insulin Delivered (U),0.395,red
85961,2024-07-01 14:44:00,bolus_Insulin Delivered (U),0.382,red
86036,2024-07-01 15:59:00,bolus_Insulin Delivered (U),0.304,red
86301,2024-07-01 20:24:00,bolus_Insulin Delivered (U),0.427,red


,DateTime,Series,Value,Color
43936,2024-06-02 12:17:00,bolus_Insulin Delivered (U),0.400,red
44172,2024-06-02 16:13:00,bolus_Insulin Delivered (U),0.547,red
44368,2024-06-02 19:29:00,bolus_Insulin Delivered (U),0.389,red
45344,2024-06-03 11:42:00,bolus_Insulin Delivered (U),0.326,red
45383,2024-06-03 12:21:00,bolus_Insulin Delivered (U),0.410,red
...,...,...,...,...
85737,2024-07-01 11:00:00,bolus_Insulin Delivered (U),0.395,red
85961,2024-07-01 14:44:00,bolus_Insulin Delivered (U),0.382,red
86036,2024-07-01 15:59:00,bolus_Insulin Delivered (U),0.304,red
86301,2024-07-01 20:24:00,bolus_Insulin Delivered (U),0.427,red


,DateTime,Series,Value,Color
43936,2024-06-02 12:17:00,bolus_Insulin Delivered (U),0.400,red
44172,2024-06-02 16:13:00,bolus_Insulin Delivered (U),0.547,red
44368,2024-06-02 19:29:00,bolus_Insulin Delivered (U),0.389,red
45344,2024-06-03 11:42:00,bolus_Insulin Delivered (U),0.326,red
45383,2024-06-03 12:21:00,bolus_Insulin Delivered (U),0.410,red
...,...,...,...,...
85737,2024-07-01 11:00:00,bolus_Insulin Delivered (U),0.395,red
85961,2024-07-01 14:44:00,bolus_Insulin Delivered (U),0.382,red
86036,2024-07-01 15:59:00,bolus_Insulin Delivered (U),0.304,red
86301,2024-07-01 20:24:00,bolus_Insulin Delivered (U),0.427,red


,DateTime,Series,Value,Color
6517,2024-06-02 12:17:00,bolus_Insulin Delivered (U),0.400,red
6753,2024-06-02 16:13:00,bolus_Insulin Delivered (U),0.547,red
6949,2024-06-02 19:29:00,bolus_Insulin Delivered (U),0.389,red
7925,2024-06-03 11:42:00,bolus_Insulin Delivered (U),0.326,red
7964,2024-06-03 12:21:00,bolus_Insulin Delivered (U),0.410,red
8301,2024-06-03 17:58:00,bolus_Insulin Delivered (U),0.270,red
9406,2024-06-04 12:12:00,bolus_Insulin Delivered (U),0.377,red
9566,2024-06-04 14:52:00,bolus_Insulin Delivered (U),0.310,red
9831,2024-06-04 19:17:00,bolus_Insulin Delivered (U),0.450,red
9896,2024-06-04 20:22:00,bolus_Insulin Delivered (U),0.143,red


,Time,Median,Q1,Q3,D1,D9,Min,Max
0,00:00:00,9.86,8.4400,11.050,7.068,11.900,5.2,13.20
1,00:01:00,9.70,8.2825,10.520,6.852,11.822,5.3,13.22
2,00:02:00,9.60,8.3300,10.505,6.928,11.946,5.4,13.24
3,00:03:00,9.50,8.3775,10.490,7.004,11.894,5.5,13.26
4,00:04:00,9.40,8.4250,10.475,7.080,11.842,5.6,13.28
...,...,...,...,...,...,...,...,...
1435,23:55:00,9.88,8.4100,10.800,6.972,12.016,5.2,13.36
1436,23:56:00,9.86,8.4200,10.720,6.904,11.780,5.2,13.40
1437,23:57:00,9.84,8.3400,10.640,6.836,11.624,5.2,13.30
1438,23:58:00,9.82,8.2200,10.620,6.768,11.596,5.2,13.20


In [13]:
import plotly.graph_objects as go

# Initialize the Dash app
app = Dash(__name__)


# Create a line plot using Plotly
def create_plot():
    fig = go.Figure()

    # Add median line
    fig.add_trace(go.Scatter(
        x=df_glucose['Time'],
        y=df_glucose['Median'],
        mode='lines',
        name='Median',
        line=dict(color='green')
    ))

    # Add first quartile (Q1) line
    fig.add_trace(go.Scatter(
        x=df_glucose['Time'],
        y=df_glucose['Q1'],
        mode='lines',
        name='Q1 and Q3',
        line=dict(color='orange', dash='dot'),
        showlegend=False  # Hide legend for this trace
    ))

    # Add third quartile (Q3) line
    fig.add_trace(go.Scatter(
        x=df_glucose['Time'],
        y=df_glucose['Q3'],
        mode='lines',
        line=dict(color='orange', dash='dot'),
        fill='tonexty',  # This fills the area between the Q1 and Q3 lines
        fillcolor='rgba(255, 165, 0, 0.3)',  # Set fill color with transparency
        name='Q1 and Q3'  # Use same label for legend
    ))

    # Add first quartile (D1) line
    fig.add_trace(go.Scatter(
        x=df_glucose['Time'],
        y=df_glucose['D1'],
        mode='lines',
        name='D1 and D9',
        line=dict(color='blue', dash='dot'),
        showlegend=False  # Hide legend for this trace
    ))

    # Add third quartile (D9) line
    fig.add_trace(go.Scatter(
        x=df_glucose['Time'],
        y=df_glucose['D9'],
        mode='lines',
        line=dict(color='blue', dash='dot'),
        fill='tonexty',  # This fills the area between the Q1 and Q3 lines
        fillcolor='rgba(0, 165, 255, 0.1)',  # Set fill color with transparency
        name='D1 and D9'  # Use same label for legend
    ))

    # Add first quartile (D1) line
    fig.add_trace(go.Scatter(
        x=df_glucose['Time'],
        y=df_glucose['Max'],
        mode='lines',
        name='Min and Max',
        line=dict(color='yellow', dash='dash'),
        showlegend=False  # Hide legend for this trace,
    ))

    # Add third quartile (D9) line
    fig.add_trace(go.Scatter(
        x=df_glucose['Time'],
        y=df_glucose['Min'],
        mode='lines',
        line=dict(color='yellow', dash='dash'),
        name='Min and Max'  # Use same label for legend
    ))

    # Update layout
    fig.update_layout(
        title='Summary Statistics by Time of Day',
        xaxis_title='Time of Day',
        yaxis_title='Value',
        xaxis=dict(tickmode='array'),
        legend=dict(x=0.01, y=0.99)
    )

    return fig


# Define the layout of the app
app.layout = html.Div([
    html.H1("Time Series Summary Statistics"),
    dcc.Graph(figure=create_plot())  # Use the create_plot function to generate the figure
])

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)

In [13]:
df_basal_insulin = SummarizeMetric(df, 'basal_Insulin Delivered (U)')
display(df_basal_insulin)

,Time,Median,Q1,Q3,D1,D9,Min,Max
0,00:00:00,0.006667,0.006667,0.006875,0.006667,0.013658,0.002533,0.018817
1,00:01:00,0.006667,0.006667,0.006667,0.006253,0.013292,0.000000,0.018817
2,00:02:00,0.006667,0.006667,0.007500,0.006357,0.013907,0.000000,0.018817
3,00:03:00,0.007083,0.006667,0.007500,0.006357,0.016892,0.000000,0.019450
4,00:04:00,0.007500,0.006667,0.007500,0.006357,0.016892,0.000000,0.019450
...,...,...,...,...,...,...,...,...
1435,23:55:00,0.006667,0.006667,0.006667,0.006667,0.011453,0.000000,0.018817
1436,23:56:00,0.006667,0.006667,0.006667,0.006667,0.011453,0.000000,0.018817
1437,23:57:00,0.006667,0.006667,0.006667,0.006667,0.011453,0.002533,0.018817
1438,23:58:00,0.006667,0.006667,0.006667,0.006667,0.014163,0.002533,0.018817


In [14]:
df_bolus_insulin = SummarizeMetric(df, 'bolus_Insulin Delivered (U)')
display(df_bolus_insulin)

,Time,Median,Q1,Q3,D1,D9,Min,Max
0,00:00:00,0.0,0.0,0.0,0.0,0.0,0.0,1.45
1,00:01:00,0.0,0.0,0.0,0.0,0.0,0.0,0.00
2,00:02:00,0.0,0.0,0.0,0.0,0.0,0.0,0.00
3,00:03:00,0.0,0.0,0.0,0.0,0.0,0.0,0.00
4,00:04:00,0.0,0.0,0.0,0.0,0.0,0.0,0.00
...,...,...,...,...,...,...,...,...
1435,23:55:00,0.0,0.0,0.0,0.0,0.0,0.0,1.10
1436,23:56:00,0.0,0.0,0.0,0.0,0.0,0.0,0.00
1437,23:57:00,0.0,0.0,0.0,0.0,0.0,0.0,0.00
1438,23:58:00,0.0,0.0,0.0,0.0,0.0,0.0,0.00


In [36]:
display(df)

,basal_Insulin Delivered (U),bolus_Insulin Delivered (U),bolus_Trigger,cgm_CGM Glucose Value (mmol/l),Time
DateTime,,,,,
2024-06-02 00:00:00,0.006667,0.0,Manual,NaN,00:00:00
2024-06-02 00:01:00,0.006667,0.0,Manual,10.00,00:01:00
2024-06-02 00:02:00,0.006667,0.0,Manual,9.96,00:02:00
2024-06-02 00:03:00,0.006667,0.0,Manual,9.92,00:03:00
2024-06-02 00:04:00,0.006667,0.0,Manual,9.88,00:04:00
...,...,...,...,...,...
2024-07-01 21:50:00,0.013383,0.0,Manual,10.50,21:50:00
2024-07-01 21:51:00,0.013383,0.0,Manual,10.50,21:51:00
2024-07-01 21:52:00,0.013383,0.0,Manual,10.50,21:52:00


In [40]:
df_autoXmanual = df.groupby("Time")["bolus_Trigger"].max
display(df_autoXmanual)

<bound method GroupBy.max of <pandas.core.groupby.generic.SeriesGroupBy object at 0x0000017B34308CA0>>

In [ ]:
import plotly.graph_objects as go

df_autoXmanual = df.groupby("Time").max("bolus_Trigger")
display(df_autoXmanual)

# Initialize the Dash app
app = Dash(__name__)


# Create a line plot using Plotly
def create_plot():
    fig = go.Figure()

    # Add median line
    fig.add_trace(go.Scatter(
        x=df_basal_insulin['Time'],
        y=df_basal_insulin['Median'],
        mode='lines',
        name='Median',
        line=dict(color='green')
    ))

    # Add first quartile (Q1) line
    fig.add_trace(go.Scatter(
        x=df_basal_insulin['Time'],
        y=df_basal_insulin['Q1'],
        mode='lines',
        name='Q1 and Q3',
        line=dict(color='orange', dash='dot'),
        showlegend=False  # Hide legend for this trace
    ))

    # Add third quartile (Q3) line
    fig.add_trace(go.Scatter(
        x=df_basal_insulin['Time'],
        y=df_basal_insulin['Q3'],
        mode='lines',
        line=dict(color='orange', dash='dot'),
        fill='tonexty',  # This fills the area between the Q1 and Q3 lines
        fillcolor='rgba(255, 165, 0, 0.3)',  # Set fill color with transparency
        name='Q1 and Q3'  # Use same label for legend
    ))

    # Segment bolus line based on threshold, but hide individual segments from legend
    for i in range(len(df_bolus_insulin) - 1):
        x_segment = [df_bolus_insulin['Time'].iloc[i], df_bolus_insulin['Time'].iloc[i + 1]]
        y_segment = [df_bolus_insulin['Max'].iloc[i], df_bolus_insulin['Max'].iloc[i + 1]]

        # Choose color based on threshold
        #color = 'red' if df_bolus_insulin['Max'].iloc[i] >= threshold else 'blue'
        df_bolus_insulin.join(df.set_index("Time"))
        color = 'red' if df_bolus_insulin['Max'].iloc[i] else 'blue'

        # Add segment as a separate trace, without showing it in the legend
        fig.add_trace(go.Scatter(
            x=x_segment,
            y=y_segment,
            mode='lines',
            yaxis="y2",
            line=dict(color=color, width=2),
            showlegend=False
        ))

    # Add a "dummy" trace for the legend
    fig.add_trace(go.Scatter(
        x=[None],  # No actual points
        y=[None],
        mode='lines',
        name='Bolus Insulin',
        line=dict(color='blue')  # Use one of the colors to represent the legend item
    ))

    # Update layout
    fig.update_layout(
        title='Summary Statistics by Time of Day',
        xaxis_title='Time of Day',
        yaxis_title='Basal',
        yaxis2=dict(
            title='Bolus',
            overlaying='y',
            side='right',
            showgrid=False  # Hide grid for secondary y-axis if desired
        ),
        xaxis=dict(tickmode='array'),
        legend=dict(x=0.01, y=0.99)
    )

    return fig


# Define the layout of the app
app.layout = html.Div([
    html.H1("Time Series Summary Statistics"),
    dcc.Graph(figure=create_plot())  # Use the create_plot function to generate the figure
])

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)

,basal_Insulin Delivered (U),bolus_Insulin Delivered (U),cgm_CGM Glucose Value (mmol/l)
Time,,,
00:00:00,0.018817,1.45,13.20
00:01:00,0.018817,0.00,13.22
00:02:00,0.018817,0.00,13.24
00:03:00,0.019450,0.00,13.26
00:04:00,0.019450,0.00,13.28
...,...,...,...
23:55:00,0.018817,1.10,13.36
23:56:00,0.018817,0.00,13.40
23:57:00,0.018817,0.00,13.30


KeyboardInterrupt: 

In [ ]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from dash import Dash, dcc, html
from dash.dependencies import Input, Output

# Sample data generation
np.random.seed(0)  # For reproducibility
date_range = pd.date_range(start="2023-10-01", end="2023-10-07", freq='T')  # One week of minute data
data = {
    'basal_Insulin Delivered (U)': np.random.rand(len(date_range)) * 10,  # Random values between 0 and 10
    'bolus_Insulin Delivered (U)': np.random.rand(len(date_range)) * 15,
    'bolus_Trigger': np.random.choice(['Automatic', 'Manual'], size=len(date_range)),
    # Randomly choosing 'Automatic' or 'Manual'
}
df = pd.DataFrame(data, index=date_range)

# Specify the series to plot directly in the code
selected_series = ['basal_Insulin Delivered (U)', 'bolus_Insulin Delivered (U)']  # Choose specific columns to plot

# Initialize the Dash app
app = Dash(__name__)

# Layout of the Dash app
app.layout = html.Div([
    html.H1("Pre-Selected Time Series Line Plot with Annotations"),

    # Date picker for selecting date range
    dcc.DatePickerRange(
        id='date-range-picker',
        start_date=df.index.min(),
        end_date=df.index.max()
    ),

    # Line plot
    dcc.Graph(id='line-plot')
])


# Callback to update the line plot based on selected date range
@app.callback(
    Output('line-plot', 'figure'),
    Input('date-range-picker', 'start_date'),
    Input('date-range-picker', 'end_date')
)
def update_line_plot(start_date, end_date):
    # Filter data based on selected date range and specific series
    filtered_df = df.loc[start_date:end_date, selected_series + ['bolus_Trigger']]

    # Create a figure
    fig = go.Figure()

    # Add basal Insulin line
    fig.add_trace(go.Scatter(
        x=filtered_df.index,
        y=filtered_df['basal_Insulin Delivered (U)'],
        mode='lines',
        name='Basal Insulin',
        line=dict(color='blue')
    ))

    # # Add bolus Insulin line
    # fig.add_trace(go.Scatter(
    #     x=filtered_df.index,
    #     y=filtered_df['bolus_Insulin Delivered (U)'],
    #     mode='lines',
    #     name='Bolus Insulin',
    #     line=dict(color='orange')
    # ))

    # Add annotations for Automatic bolus_Trigger
    for i in range(len(filtered_df)):
        if filtered_df['bolus_Trigger'].iloc[i] == 'Automatic':
            fig.add_annotation(
                x=filtered_df.index[i],
                y=filtered_df['bolus_Insulin Delivered (U)'].iloc[i] + 1,  # Adjust y position for the annotation
                text='Automatic',  # The text for the annotation
                showarrow=True,
                arrowhead=2,
                ax=0,
                ay=-40,  # Position of the arrow from the text
                font=dict(color='red'),
                bgcolor='white',
                bordercolor='red',
                borderwidth=1,
                borderpad=4,
                opacity=0.8
            )

    # Update layout
    fig.update_layout(
        title="Time Series Line Plot with Annotations for Automatic Trigger",
        xaxis_title="Date",
        yaxis_title="Value",
        showlegend=True
    )

    return fig


# Run the Dash app
if __name__ == '__main__':
    app.run_server(debug=True)

C:\Users\Ondre\AppData\Local\Temp\ipykernel_15892\309222234.py:9: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  date_range = pd.date_range(start="2023-10-01", end="2023-10-07", freq='T')  # One week of minute data
